In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

# Path to the ZIP file
#zip_file_path = 'drive/MyDrive/archive (54).zip'
extracted_path = 'drive/MyDrive/Simpson2'

# Create the directory if it doesn't exist
#os.makedirs(extracted_path, exist_ok=True)

# Extract the ZIP file
#with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    #zip_ref.extractall(extracted_path)

#print(f'Dataset extracted to {extracted_path}')


In [ ]:
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt


Cloning into 'yolov5'...
remote: Enumerating objects: 16625, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 16625 (delta 58), reused 66 (delta 30), pack-reused 16522
Receiving objects: 100% (16625/16625), 15.16 MiB | 20.51 MiB/s, done.
Resolving deltas: 100% (11415/11415), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 778.4/778.4 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-no

In [ ]:
!pip install opencv-python-headless
!pip install torch torchvision


ERROR: Operation cancelled by user


In [ ]:
import os

# Generate the class names in a YAML-friendly format
class_names = "\n".join([f"  - {cls}" for cls in classes])

# Define the dataset YAML content
data_yaml = f"""
train: {os.path.join(output_dir, 'train.txt')}
val: {os.path.join(output_dir, 'test.txt')}
nc: {len(classes)}
names:
{class_names}
"""

# Write the YAML content to the dataset.yaml file
with open(os.path.join(output_dir, 'dataset.yaml'), 'w') as f:
    f.write(data_yaml)



In [ ]:
!python yolov5/train.py --img 416 --batch 16 --epochs 1 --data '/content/drive/MyDrive/Simpson2/output/dataset.yaml' --cfg yolov5/models/yolov5s.yaml --weights '' --name yolo_simpsons


2024-05-26 07:32:08.391209: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-26 07:32:08.391278: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-26 07:32:08.393634: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
train: weights=, cfg=yolov5/models/yolov5s.yaml, data=/content/drive/MyDrive/Simpson2/output/dataset.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=1, batch_size=16, imgsz=416, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, i

In [ ]:
!python train.py --img 416 --batch 16 --epochs 10 --data {'/content/drive/MyDrive/Simpson2/output/dataset.yaml'} --cfg models/yolov5s.yaml --weights '' --name yolo_simpsons


python3: can't open file '/content/train.py': [Errno 2] No such file or directory


In [ ]:
import os
import glob
import random
import cv2
extracted_dir_path = '/content/drive/MyDrive/Simpson2'
# Paths
train_dir = '/content/drive/MyDrive/Simpson2/train'
output_dir = os.path.join(extracted_dir_path, 'output')

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get list of class names
classes = [d for d in os.listdir(train_dir) if os.path.isdir(os.path.join(train_dir, d)) and os.listdir(os.path.join(train_dir, d))]
with open(os.path.join(output_dir, 'classes.txt'), 'w') as f:
    for cls in classes:
        f.write(cls + '\n')

# Create annotation files
def create_annotation_files():
    for cls_index, cls in enumerate(classes):
        img_dir = os.path.join(train_dir, cls)
        img_files = glob.glob(img_dir + '/*.jpg')
        random.shuffle(img_files)
        test_size = int(0.2 * len(img_files))  # 20% for testing
        test_files = img_files[:test_size]
        train_files = img_files[test_size:]
        for img_file in train_files:
            img = cv2.imread(img_file)
            height, width, _ = img.shape
            annotation = f"{cls_index} 0.5 0.5 1.0 1.0\n"
            with open(img_file.replace('.jpg', '.txt'), 'w') as f:
                f.write(annotation)
        with open(os.path.join(output_dir, 'test.txt'), 'a') as f:
            for img_file in test_files:
                f.write(img_file + '\n')

create_annotation_files()

# Create training list
def create_file_list(file_list_path, img_dir):
    with open(file_list_path, 'w') as f:
        for cls in classes:
            for img_file in glob.glob(os.path.join(img_dir, cls) + '/*.jpg'):
                f.write(img_file + '\n')

create_file_list(os.path.join(output_dir, 'train.txt'), train_dir)


In [ ]:
import torch
import numpy as np
from yolov5.models.experimental import attempt_load
from yolov5.utils.general import non_max_suppression, scale_coords
from yolov5.utils.datasets import letterbox
from yolov5.utils.plots import plot_one_box

# Load the model
model = attempt_load('runs/train/yolo_simpsons/weights/best.pt', map_location='cuda')
model.eval()

def detect_objects_in_video(video_path, output_path, model):
    cap = cv2.VideoCapture(video_path)
    out = None

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Preprocess the frame
        img = letterbox(frame, new_shape=416)[0]
        img = img[..., ::-1].transpose(2, 0, 1)  # BGR to RGB, HWC to CHW
        img = np.ascontiguousarray(img, dtype=np.float32)
        img /= 255.0

        # Inference
        with torch.no_grad():
            pred = model(torch.from_numpy(img).unsqueeze(0).cuda())[0]

        # Apply NMS
        pred = non_max_suppression(pred, 0.25, 0.45)[0]

        # Process detections
        if pred is not None:
            pred[:, :4] = scale_coords(img.shape[1:], pred[:, :4], frame.shape).round()
            for *xyxy, conf, cls in pred:
                label = f'{classes[int(cls)]} {conf:.2f}'
                plot_one_box(xyxy, frame, label=label, color=(255, 0, 0), line_thickness=2)

        # Write frame to output video
        if out is None:
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            out = cv2.VideoWriter(output_path, fourcc, 20.0, (frame.shape[1], frame.shape[0]))
        out.write(frame)

    cap.release()
    out.release()

# Example usage
video_path = os.path.join(extracted_dir_path, 'simpsons_video.mp4')
output_path = os.path.join(extracted_dir_path, 'simpsons_output.avi')
detect_objects_in_video(video_path, output_path, model)
